# Clasifying Tweet sentiments


© Explore Data Science Academy

In [1]:
# import relevant libraries
import nltk
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
pd.set_option('display.max_rows', 100)

from nltk.corpus import stopwords
from sklearn.metrics import classification_report

# set plot style
sns.set()

In [2]:
# Loading Data
df = pd.read_csv('C:/Users/Mpilenhle/Documents/EDSA/Classification/Advanced_Classification_Predict-student_data-2780/train.csv')
df_test = pd.read_csv('C:/Users/Mpilenhle/Documents/EDSA/Classification/Advanced_Classification_Predict-student_data-2780/test_with_no_labels.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Mpilenhle/Documents/EDSA/Classification/Advanced_Classification_Predict-student_data-2780/train.csv'

In [ ]:
# looking at the data
df.head(15)

In [ ]:
df[df['sentiment'] == -1]

In [ ]:
df.shape

In [ ]:
df_test.head()

In [ ]:
df_test.shape

In [ ]:
#looking at the different types of labels
type_labels = list(df.sentiment.unique())
print(type_labels)

In [ ]:
# visualising the data distribution
df['sentiment'].value_counts().plot(kind = 'bar')
plt.show()

# Removing Noise from the data

In [ ]:
#remmoving the urls
pattern_url = r'http[s]?://t.co/[A-Za-z0-9]+'
subs_url = r'url-web'
df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
v = df['message']
c = [i for i in v]
for i in range(50):
    print(str(i) +')' + c[i] +'\n')

In [ ]:
#remmoving the Retweets
pattern_url = r'RT\s\@[A-Za-z0-9_]+:'
subs_url = r''
df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
#remmoving the mentions 
pattern_url = r'@[A-Za-z0-9_]+'
subs_url = r''
df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
#remmoving the Hashtags 
pattern_url = r'\#[A-Za-z0-9#?_]+'
subs_url = r''
df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
#remmoving the remaining https
pattern_url = r'https:[.*?]+'
subs_url = r''
df['message'] = df['message'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [ ]:
# turning all tweets to lower case
df['message'] = df['message'].str.lower()

In [ ]:
# punctuation remover function
import string
def remove_punctuation(tweets):
    return ''.join([l for l in tweets if l not in string.punctuation])

In [ ]:
# using apply method to remove the punctuation marks
df['message'] = df['message'].apply(remove_punctuation)

In [ ]:
#creating a function for removing emojis
import re
def remove_emoji(string):
    emoji_pattern = re.compile("[" 
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
# Removing the emojis using the apply method
df['message'] = df['message'].apply(remove_emoji)

# Tokenizing the tweets

In [ ]:
# importing tokenizing library
from nltk.tokenize import word_tokenize, TreebankWordTokenizer

In [ ]:
tokeniser = TreebankWordTokenizer()
df['tokens'] = df['message'].apply(tokeniser.tokenize)

In [ ]:
df.head()

# Lemmatizing the tweets

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [ ]:
# lemmatizing function
def tweet_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words if word.isalpha()]    

In [ ]:
df['lemma'] = df['tokens'].apply(tweet_lemma, args=(lemmatizer, ))

In [ ]:
df.head()

# Stemming the Lemma Tokens

In [ ]:
#importing stemmer library
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer

In [ ]:
#Stemmer function
def token_stemmer(words, stemmer):
    return [stemmer.stem(word) for word in words]

In [ ]:
# find the stem of each word in the original tokens
stemmer = SnowballStemmer('english')
df['original_stem'] = df['tokens'].apply(token_stemmer, args=(stemmer, ))

In [ ]:
# find the stem of each word in the Lemma tokens
stemmer = SnowballStemmer('english')
df['lemma_stem'] = df['lemma'].apply(token_stemmer, args=(stemmer, ))

In [ ]:
df.lemma_stem.head()

# Removing the stop words

In [ ]:
def remove_stop_words(tokens):    
    return [t for t in tokens if t not in stopwords.words('english')]

In [ ]:
df['lemma_no_stop_words'] = df['lemma_stem'].apply(remove_stop_words)

In [ ]:
df['original_no_stop_words'] = df['lemma_stem'].apply(remove_stop_words)

In [ ]:
df.head()

# Creating Ngrams

In [ ]:
'''
The function creates a new data frame with 2 ngrams to test
returns a function with a new column of ngrams

'''
from nltk.util import ngrams
def ngramconvert(df,n=2):
    for item in df.columns:
        df['new_'+item]=df[item].apply(lambda sentence: list(ngrams(sentence.split(), n)))
    return df

In [ ]:
ngram_df = ngramconvert(df[['message']],2)
ngram_df.head()

# Creating a count vector with 2 ngrams not scaled


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
# splitting the data
X = df['message']
y = df['sentiment']

In [ ]:
# splitting the data
X_train1, X_test1, y_train, y_test = train_test_split(X, y,test_size=0.2,
random_state=53)

In [ ]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2))
ngram_count_train1 = count_vectorizer.fit_transform(X_train1.values)
ngram_count_test1 = count_vectorizer.transform(X_test1.values)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB()
nb_classifier.fit(ngram_count_train1, y_train)

ngram_pred = nb_classifier.predict(ngram_count_test1)
metrics.accuracy_score(y_test, ngram_pred)

In [ ]:
print('2 ngram  Model')
print(classification_report(y_test, ngram_pred, target_names= ['-1', '0', '1', '2']))

In [ ]:
#traing the model with the entire data
yf = df['sentiment']
Xf = df['message']

In [ ]:
X_test = df_test['message']

In [ ]:
count_vectorizerf = CountVectorizer(ngram_range=(1, 2))
count_finalf = count_vectorizerf.fit_transform(Xf)
count_final_test = count_vectorizerf.transform(X_test)

In [ ]:
# Import the scaler module
from sklearn import preprocessing

fscaler = preprocessing.MaxAbsScaler()
f_scaled = fscaler.fit_transform(count_finalf)
f_scaled_test = fscaler.transform(count_final_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

final_classifier2 = MultinomialNB()
final_classifier2.fit(f_scaled, yf)


In [ ]:
final_preds = final_classifier.predict(count_final_test)

In [ ]:
final_preds = final_classifier.predict(count_final_test)
daf=pd.DataFrame(final_preds, columns=['sentiment'])
daf.head()

output = pd.DataFrame({"tweetid":df_test['tweetid']})
submission3 = output.join(daf)        
submission3.to_csv("submission3.csv", index=False)

In [ ]:
output = pd.DataFrame({"tweetid":df_test['tweetid']})
submission3 = output.join(daf)        
submission3.to_csv("submission3.csv", index=False)

# using  The cleaned column



In [ ]:
clean_sentences = [" ".join(i) for i in df['lemma_no_stop_words']]
df['clean_sentences'] = clean_sentences
df.head()

In [ ]:
x_2 = df['clean_sentences']
y_2 = df['sentiment']

In [ ]:
x_2.head()

In [ ]:
# splitting the data
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(x_2, y_2,test_size=0.2,
random_state=53)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

vectorizer = TfidfVectorizer()
vectorised_train_documents = vectorizer.fit_transform(X_train_2)
vectorised_test_documents = vectorizer.transform(X_test_2)

In [ ]:
from yellowbrick.text import FreqDistVisualizer
features = vectorizer.get_feature_names()
visualizer = FreqDistVisualizer(features=features, orient='v', n = 25)
visualizer.fit(vectorised_train_documents)
visualizer.show()

In [ ]:
from yellowbrick.text import UMAPVisualizer

umap = UMAPVisualizer(metric="cosine")
umap.fit(vectorised_train_documents)
umap.show()

In [ ]:
from yellowbrick.regressor import ResidualsPlot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1
)

visualizer = ResidualsPlot(LinearRegression())
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.show()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB()
nb_classifier.fit(vectorised_train_documents, y_train_2)

bag_pred = nb_classifier.predict(vectorised_test_documents)
metrics.accuracy_score(y_test_2, bag_pred)

In [ ]:
print('bag of words')
print(classification_report(y_test_2, bag_pred, target_names= ['-1', '0', '1', '2']))

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(vectorised_train_documents, y_train_2)
clf.score(vectorised_test_documents, y_test_2)

In [ ]:
b_pred = clf.predict(vectorised_test_documents)

In [ ]:
print(classification_report(y_test_2, b_pred, target_names= ['-1', '0', '1', '2']))

In [ ]:
count_fnal = 

final_pred = clf.predict(count_final_test)
daf=pd.DataFrame(final_pred, columns=['sentiment'])
daf.head()

output = pd.DataFrame({"tweetid":df_test['tweetid']})
submission4 = output.join(daf)        
submission4.to_csv("submission4.csv", index=False)

In [ ]:
# Import the scaler module
from sklearn import preprocessing

fscaler = preprocessing.MaxAbsScaler()
scaled_train = fscaler.fit_transform(vectorised_train_documents)
scaled_test = fscaler.transform(vectorised_test_documents)

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(vectorised_train_documents, y_train_2)
clf.score(scaled_test, y_test_2)

In [ ]:
s_pred = clf.predict(scaled_test)

In [ ]:
print(classification_report(y_test_2, s_pred, target_names= ['-1', '0', '1', '2']))

In [ ]:
vectorised_train_documents
vectorised_test_documents